# Autonomous Driving Object Detection and Recognition System.
#### Using [yolov8](https://docs.ultralytics.com/models/yolov8/) object detection algorithms.
#### Dataset from [bdd100k](https://doc.bdd100k.com/index.html).
#### Author: Samir K C

In [1]:
# Setting up my environment

#Verify Python Environment
import sys
print(f"Python version: {sys.version}")
print(f"Python location: {sys.executable}")

# Verify PyTorch installation and MPS (Metal Performance Shaders) availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"MPS (Apple Metal) available: {torch.backends.mps.is_available()}")
print(f"Current PyTorch device: {torch.device('mps' if torch.backends.mps.is_available() else 'cpu')}")

# Verify other essential packages
import cv2
print(f"OpenCV version: {cv2.__version__}")

from ultralytics import YOLO
print("Ultralytics is installed.")

#Set default device for PyTorch
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

Python version: 3.9.20 | packaged by conda-forge | (main, Sep 30 2024, 17:48:00) 
[Clang 17.0.6 ]
Python location: /Users/samirkc/micromamba/envs/yolov8-env/bin/python3.9
PyTorch version: 2.5.1
MPS (Apple Metal) available: True
Current PyTorch device: mps
OpenCV version: 4.10.0
Ultralytics is installed.
Using device: mps


In [2]:
# Define paths
from pathlib import Path

DATASET_PATH = Path('./datasets/coco_splitted/') 
WEIGHTS_PATH = Path('./weights/')  # Path to save model weights
RESULTS_PATH = Path('./results')  # Path to save results

# Create necessary directories
WEIGHTS_PATH.mkdir(exist_ok=True)
RESULTS_PATH.mkdir(exist_ok=True)


In [3]:
# Training Parameters
params = {
    # Data Parameters
    'data_yaml': str(DATASET_PATH / 'data.yaml'),
    'img_size': 320,         # Reduced image size for faster training
    'batch_size': 4,        # Moderate batch size; increase if memory allows
    
    # Training Parameters
    'epochs': 12,            # Cap epochs, or lower if accuracy is met early
    'patience': 3,           # Early stopping patience for quicker training end
    'model_type': 'yolov8n.pt', 
    'device': 'mps',         # Optimized for Apple M3 Pro

    # Optimizer Parameters
    'optimizer': 'auto',     # Let YOLOv8 auto-select the optimizer
    'lr0': 0.01,             # Initial learning rate for efficient convergence
    'lrf': 0.15,             # Higher learning rate decay for faster convergence
    'momentum': 0.937,       # Momentum value
    'weight_decay': 0.0005,
    'warmup_epochs': 1.0,    # Reduced warmup for faster adaptation
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,

    # Augmentation Parameters (disabled to save time)
    # 'hsv_h': 0.015,
    # 'hsv_s': 0.5,            # Slightly reduced saturation variation
    # 'hsv_v': 0.2,            # Reduced brightness variation
    # 'degrees': 0.0,
    # 'translate': 0.1,
    # 'scale': 0.5,
    # 'shear': 0.0,
    # 'flipud': 0.0,
    # 'fliplr': 0.5,
    # 'mosaic': 0.1,           # Lower mosaic probability to reduce training time
    # 'mixup': 0.0,            # Disabled mixup to speed up training

    # Other Parameters
    'save_period': -1,       # Save checkpoint at the final epoch
    'cache': False,          # Do not cache images to save memory
    'half': True,            # Mixed precision for faster training
    'accum': 2,              # Gradient accumulation to simulate larger batch size
}


In [4]:
from tqdm import tqdm
import time

def train_model(params):
    """Train YOLOv8 model with ETA tracking and batch progress"""
    # Load model
    model = YOLO(params['model_type'])

    total_epochs = params['epochs']
    batch_size = params['batch_size']
    
    # Tracking overall training time
    start_time = time.time()
    
    for epoch in range(total_epochs):
        epoch_start_time = time.time()
        
        print(f"\nStarting Epoch {epoch + 1}/{total_epochs}")

        # Adding tqdm for batch progress within the epoch
        batch_progress = tqdm(
            model.train(
                data=params['data_yaml'],
                imgsz=params['img_size'],
                epochs=1,   # Run one epoch at a time to capture progress
                batch=batch_size,
                device=params['device'],
                optimizer=params['optimizer'],
                lr0=params['lr0'],
                lrf=params['lrf'],
                momentum=params['momentum'],
                weight_decay=params['weight_decay'],
                warmup_epochs=params['warmup_epochs'],
                warmup_momentum=params['warmup_momentum'],
                warmup_bias_lr=params['warmup_bias_lr'],
                patience=params['patience'],
                save_period=params['save_period'],
                cache=params['cache'],
                project=str(RESULTS_PATH),
                name='train'
            ),
            desc=f"Epoch {epoch + 1}/{total_epochs}",
            unit="batch"
        )

        # Calculating time per epoch and estimated remaining time
        epoch_time = time.time() - epoch_start_time
        total_elapsed = time.time() - start_time
        estimated_remaining = epoch_time * (total_epochs - (epoch + 1))
        
        print(f"Epoch {epoch + 1}/{total_epochs} completed in {epoch_time:.2f}s")
        print(f"Estimated remaining time: {estimated_remaining // 60:.0f} min {estimated_remaining % 60:.0f} sec")
        print(f"Total elapsed time: {total_elapsed // 60:.0f} min {total_elapsed % 60:.0f} sec")
        
    # Return model and results after all epochs are done
    total_training_time = time.time() - start_time
    print(f"\nTraining complete in {total_training_time // 60:.0f} min {total_training_time % 60:.0f} sec")

    return model, batch_progress


In [5]:
# Evaluate Models
def evaluate_model(model):
    """Comprehensive model evaluation"""
    # Run validation on test set
    metrics = model.val(data=params['data_yaml'])
    
    # Extract metrics
    evaluation = {
        'mAP50': metrics.box.map50,    # mAP at IoU=0.5
        'mAP50-95': metrics.box.map,    # mAP at IoU=0.5:0.95
        'precision': metrics.box.mp,     # mean precision
        'recall': metrics.box.mr,        # mean recall
        'f1-score': 2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr),
        
        # Per-class metrics
        'per_class_mAP50': metrics.box.ap50_per_class,
        'per_class_precision': metrics.box.p_per_class,
        'per_class_recall': metrics.box.r_per_class
    }
    
    # Save metrics
    metrics_df = pd.DataFrame({
        'Class': ['pedestrian', 'rider', 'car', 'truck', 'bus', 'train', 
                 'motorcycle', 'bicycle', 'traffic light', 'traffic sign'],
        'mAP50': evaluation['per_class_mAP50'],
        'Precision': evaluation['per_class_precision'],
        'Recall': evaluation['per_class_recall']
    })
    
    metrics_df.to_csv(RESULTS_PATH / 'class_metrics.csv', index=False)
    
    return evaluation, metrics_df

In [6]:
# Analyze Results
def analyze_results(evaluation, metrics_df):
    """Analyze and visualize evaluation results"""
    # Print overall metrics
    print("\nOverall Metrics:")
    print(f"mAP50: {evaluation['mAP50']:.4f}")
    print(f"mAP50-95: {evaluation['mAP50-95']:.4f}")
    print(f"Precision: {evaluation['precision']:.4f}")
    print(f"Recall: {evaluation['recall']:.4f}")
    print(f"F1-Score: {evaluation['f1-score']:.4f}")
    
    # Plot per-class metrics
    plt.figure(figsize=(12, 6))
    metrics_df.plot(x='Class', y=['mAP50', 'Precision', 'Recall'], kind='bar')
    plt.title('Per-class Performance Metrics')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(RESULTS_PATH / 'class_performance.png')
    plt.close()

In [8]:








# Main Execution
if __name__ == "__main__":
    # Train model
    #model, results = train_model(params)
    model =     'yolov8n.pt', 

    # Evaluate model
    evaluation, metrics_df = evaluate_model(model)
    
    # Analyze results
    analyze_results(evaluation, metrics_df)

NameError: name 'model' is not defined